<a href="https://colab.research.google.com/github/shivamsinghtomar78/LangChain/blob/main/Tool_calling_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["GOOGLE_API_KEY"] = " "

In [7]:
!pip install -q  langchain-core requests

In [8]:
!pip install langchain-google-genai

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import os

In [10]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [11]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [12]:
multiply.name

'multiply'

In [13]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [14]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

Tool Binding

In [17]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0.5)

In [19]:
llm.invoke('hello beautiful')

AIMessage(content="Hello! It's nice to hear from you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--af948b5c-e489-40f6-b374-e67575adde75-0')

In [20]:
llm_with_tools = llm.bind_tools([multiply])

In [21]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content='I am doing well, thank you for asking. How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--4e33e379-f165-4c97-80fe-39639b907d4b-0')

In [22]:
query = HumanMessage('can you multiply 3 with 1000')

In [23]:
messages = [query]

In [24]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [25]:
result = llm_with_tools.invoke(messages)

In [26]:
messages.append(result)

In [27]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--7cd93953-5c6e-4e04-a17a-7f68a49c7c19-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': 'ea56a8c1-e9d5-4ba6-abe6-3be8698180ce', 'type': 'tool_call'}])]

In [28]:
tool_result = multiply.invoke(result.tool_calls[0])

In [29]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='ea56a8c1-e9d5-4ba6-abe6-3be8698180ce')

In [30]:
messages.append(tool_result)

In [31]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--7cd93953-5c6e-4e04-a17a-7f68a49c7c19-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': 'ea56a8c1-e9d5-4ba6-abe6-3be8698180ce', 'type': 'tool_call'}]),
 ToolMessage(content='3000', name='multiply', tool_call_id='ea56a8c1-e9d5-4ba6-abe6-3be8698180ce')]

In [32]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is 3000.'

In [35]:
!pip install -q requests

In [36]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [34]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [37]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756339201,
 'time_last_update_utc': 'Thu, 28 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1756425601,
 'time_next_update_utc': 'Fri, 29 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.7256}

In [39]:
convert.invoke({'base_currency_value':5, 'conversion_rate':85.16})

425.79999999999995

tool binding

In [40]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [41]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [42]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [43]:
ai_message = llm_with_tools.invoke(messages)

In [44]:
messages.append(ai_message)

In [45]:
ai_message.tool_calls

[]

In [46]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [47]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I am sorry, I cannot fulfill this request. The available tools do not have the ability to provide conversion rates between currencies. I can only multiply two given numbers.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--7863763b-841f-4118-a1f8-921922465862-0')]

In [48]:
llm_with_tools.invoke(messages).content

''

In [49]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

/tmp/ipython-input-627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [50]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I am doing well, thank you for asking. How can I help you today?"
}
```

> Finished chain.
